In [39]:

import pandas as pd
import numpy as np
import os
import openpyxl as xl

import re

import warnings
warnings.filterwarnings('ignore')

from Utilities import FileIO
from Utilities import DataframeCleaning


# Load Files

In [40]:

df = FileIO.importFile("transformFinished_merged.csv")

# Calculate Totals (incl International)

In [41]:


keyFrame = FileIO.importKey()


keyFrame[keyFrame['mail_class']=='Summary Other'][['product_id','product']]
# keyFrame[keyFrame['mail_class']=='Summary Other'][['product_id','product']]['product_id'].to_list()


,product_id,product
100,262,Total First Class (incl International)
109,263,Total Marketing Mail (incl International)
112,265,Total Package Services (incl International)
114,264,Total Periodicals (incl International)
116,266,Total Special Services (incl International)


In [42]:
isClass = keyFrame['mail_class']=="First Class"
isFC = keyFrame[isClass]

dropProducts_fc= [3,4,8,9]

fcChildren_overall = list(isFC['product_id'])
fcChildren =[c for c in fcChildren_overall if c not in dropProducts_fc]

isMM = keyFrame[keyFrame['mail_class']=="Marketing Mail"]
mmChildren = list(isMM['product_id'])


isPER =  keyFrame[keyFrame['mail_class']=="Periodicals"]
perChildren =  list(isPER['product_id'])

isPS =  keyFrame[keyFrame['mail_class']=="Package Services"]
psChildren =  list(isPS['product_id'])


isSS =  keyFrame[keyFrame['mail_class']=="Special Services"]
ssChildren_overall =  list(isSS['product_id'])
dropproducts_ss = [108]

ssChildren = [e for e in ssChildren_overall if e not in dropproducts_ss]





summaryDict = {262: fcChildren, 
               263: mmChildren, 
               265: psChildren, 
               264: perChildren, 
               266: ssChildren}
# summaryDict


In [43]:
summaryDict_strings = {}
for key,value in summaryDict.items():
    summaryDict_strings[str(key)] = [str(e) for e in value]
# summaryDict_strings

In [44]:
df['product_id'] = df['product_id'].astype(str)

def markOriginalSums(row):
    if row in summaryDict_strings.keys():
        return row +"_orig"
    return row

df['product_id'] = df['product_id'].apply(lambda row: markOriginalSums(row))


In [45]:
def calculateTotals(frame, idToRecalculate, childrenList):
    frameOrig = frame.copy()
    frame = frame[frame['product_id']!= idToRecalculate]
    frameIdListFilter = frame['product_id'].isin(childrenList)
    valueColumns = ['volume','revenue','attributable_cost']

    grp = frame[frameIdListFilter].groupby("fy")[valueColumns].sum()
    grp.reset_index(inplace=True)

    grp['product_id'] = idToRecalculate

    for col in frameOrig.columns:
        if col not in grp:
            grp[col] = np.nan
    
    rez = pd.concat([frame, grp])

    return rez


def calculateTotals_aggregate(frame):
    for keyDict in summaryDict_strings.keys():
        summaryRow = keyDict

        childrenList = summaryDict_strings[keyDict]
        frame = calculateTotals(frame, summaryRow, childrenList)

    
    
    return frame


df = calculateTotals_aggregate(df)

# df[df['product_id']=='262']



# Identify Errors

In [46]:
def spotCheck(year, parentId, frame):
    parentIdStr = str(parentId)
    isYear = frame['fy']==year
    isSummaryId = frame['product_id'].str.contains(parentIdStr)
    isChildren = frame['product_id'].isin(summaryDict_strings[parentIdStr])
    rez = frame[isYear & (isSummaryId | isChildren)]

    def applySummaryIdentification(row):
        if str(parentId) in row:
            return True
        return False
    
    rez['summary'] = rez['product_id'].apply(lambda row: applySummaryIdentification(row) )
    
    rez = rez.sort_values(by="summary", ascending=False)
    return rez

yearToCheck = 2018
idToCheck = 262


In [47]:
def reportDiffs(year, summaryRowId):
    frame = df.copy()

    if type(summaryRowId==str):
         summaryRowId = str(summaryRowId)

    secondarySummaryRowId = summaryRowId+"_orig"
    isYear = frame['fy']==year
    isSummaryRow = frame['product_id']==summaryRowId
    isSecondarySummaryRow = frame['product_id']==secondarySummaryRowId

    summaryFrame = frame[isYear & isSummaryRow]
    # return summaryFrame

    secondarySummaryFrame = frame[isYear & isSecondarySummaryRow]
    # return secondarySummaryFrame

    combinedFrame = pd.concat([summaryFrame, secondarySummaryFrame])
    # return combinedFrame

    rez = {'fy': year, 
            'sumRow': summaryRowId}

    # rez = pd.DataFrame([rez])

    if len(combinedFrame)>1:
            for col in ['volume','revenue','attributable_cost']:
                colDiff = combinedFrame[col].diff().iloc[1]

                if (colDiff > -5 and colDiff < 5) or pd.isna(colDiff):
                    colDiff=0
                rez[col] = colDiff
    
    
    #any products have attributable cost only? 
    spotCheckFrame = spotCheck(year, summaryRowId, df)
    sp = spotCheckFrame.copy()
    acOnly = sp.loc[(sp['volume']==0)&(sp['revenue']==0)&(sp['attributable_cost']!=0)]
    if len(acOnly)==0:
         rez['attributable_cost'] = 0
                                                        


    return rez

reportDiffs(2011, 266)

{'fy': 2011,
 'sumRow': '266',
 'volume': 0,
 'revenue': 0,
 'attributable_cost': 0}

In [48]:
print(list(summaryDict_strings.keys())[1]), summaryDict_strings['263']

263


(None,
 ['26', '27', '231', '21', '22', '23', '24', '25', '104', '135', '139', '223'])

In [49]:
yearToCheck, idToCheck = 2014, 266
# originalSummary = dictKey +"_orig"
reportDiffs(2014, 266)


{'fy': 2014,
 'sumRow': '266',
 'volume': 0,
 'revenue': 0,
 'attributable_cost': 0}

In [50]:
# spotCheck(2014, 266, df)

spotCheckFrame = spotCheck(yearToCheck, idToCheck, df)
sp = spotCheckFrame.copy()
acOnly = sp.loc[(sp['volume']==0)&(sp['revenue']==0)&(sp['attributable_cost']!=0)]
if len(acOnly)==0:
    print("e")
# acOnly
#         combinedFrame['attributable_cost'] = 0

e


In [51]:
df.head()

,product_id,product,mail_class,fy,volume,revenue,attributable_cost,volume_variable_cost,product_specific_cost,rev_pc,attributable_cost_pc,cont_pc,cost_coverage,weight,weight_pc
0,3,Single Piece Letters,First Class,2008,3.350971e+10,1.435352e+10,8.443076e+09,8.434323e+09,8753000.0,0.428339,0.251959,0.176380,1.700035,1.000329e+09,0.477631
1,4,Single Piece Cards,First Class,2008,1.845860e+09,5.004896e+08,4.465088e+08,4.460268e+08,482000.0,0.271142,0.241897,0.029244,1.120895,1.183008e+07,0.102544
2,5,Total Single Piece Letters and Cards,First Class,2008,3.535557e+10,1.485401e+10,8.889584e+09,8.880349e+09,9235000.0,0.420132,0.251434,0.168698,1.670946,1.012159e+09,0.458048
3,8,Presort Letters,First Class,2008,4.837987e+10,1.632780e+10,5.441668e+09,5.429030e+09,12638000.0,0.337492,0.112478,0.225014,3.000515,2.174874e+09,0.719266
4,9,Presort Cards,First Class,2008,3.555997e+09,7.322368e+08,2.822705e+08,2.813415e+08,929000.0,0.205916,0.079379,0.126537,2.594096,2.896925e+07,0.130345


In [52]:

def calculateErrorFrame(year):
    errorFrame = pd.DataFrame()

    for dictKey in summaryDict_strings.keys():
        originalSummary = dictKey +"_orig"
        errorDict = reportDiffs(year, dictKey)
        errorRow = pd.DataFrame([errorDict])
        errorFrame = pd.concat([errorFrame, errorRow])


    return errorFrame

fullErrorFrame = pd.DataFrame()


for y in range(df.fy.min(), df.fy.max()+1):
    errorFrame_current = calculateErrorFrame(y)
    fullErrorFrame = pd.concat([fullErrorFrame, errorFrame_current])

continuousCols = ['volume','revenue','attributable_cost']

fullErrorFrame = fullErrorFrame.dropna(subset=continuousCols, how='all')
fullErrorFrame = fullErrorFrame[~((fullErrorFrame[continuousCols] > -5) & (fullErrorFrame[continuousCols] < 5)).all(axis=1)]


fullErrorFrame.rename(columns={"sumRow":"product_id"}, inplace=True)
fullErrorFrame['product_id'] = fullErrorFrame['product_id'].astype(int)
fullErrorFrame = DataframeCleaning.realignWithKey(fullErrorFrame, keyFrame)
fullErrorFrame = fullErrorFrame.sort_values(by=["fy", "product_id"])
fullErrorFrame


,product_id,product,mail_class,fy,volume,revenue,attributable_cost
0,263,Total Marketing Mail (incl International),Summary Other,2011,0,-39301.999992,0


In [53]:
df['product'] = df['product'].astype(str)

In [1]:
print(fullErrorFrame)
print('2011 total MM revenue is an error in the source data')

SyntaxError: unmatched ')' (2661416836.py, line 2)

# Spot Check Errors

In [55]:


spotCheck(yearToCheck, idToCheck, df)

,product_id,product,mail_class,fy,volume,revenue,attributable_cost,volume_variable_cost,product_specific_cost,rev_pc,attributable_cost_pc,cont_pc,cost_coverage,weight,weight_pc,summary
6,266,nan,NaN,2014,3.473255e+09,2.060094e+09,1.331263e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
305,266_orig,Total Special Services (incl International),Summary Other,2014,3.473255e+09,2.060094e+09,1.331241e+09,1.324522e+09,6.870500e+06,NaN,NaN,NaN,NaN,NaN,NaN,True
760,57,Stamped Cards,Special Services,2014,NaN,1.680520e+06,5.759984e+05,5.759984e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
767,138,Stamp Fulfillment Services,Special Services,2014,NaN,3.298493e+06,4.253758e+06,4.253758e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
766,74,Post Office Box Service,Special Services,2014,NaN,3.652866e+08,2.819354e+08,2.819134e+08,2.200000e+04,NaN,NaN,NaN,NaN,NaN,NaN,False
765,137,Customized Postage,Special Services,2014,NaN,6.000000e+05,6.920000e+04,6.920000e+04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
764,130,Change of Address Credit Card Authentication,Special Services,2014,NaN,1.511774e+07,1.262171e+06,1.262171e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
763,62,Caller Service,Special Services,2014,NaN,9.790157e+07,2.193047e+07,2.193047e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
762,61,Address Management Services,Special Services,2014,NaN,1.723012e+07,4.346440e+06,6.112698e+04,4.285313e+06,NaN,NaN,NaN,NaN,NaN,NaN,False
298,51,Certified Mail,Special Services,2014,2.125186e+08,6.873172e+08,5.422383e+08,5.422298e+08,8.503000e+03,3.234152,2.551487,0.682664,1.267555,NaN,NaN,False


# Duplicate Values

In [56]:
print("*"*88)

****************************************************************************************


In [57]:
countSer = df.groupby(["fy", "product_id"])['product'].count()
duplicateSer = countSer.loc[countSer>1]
if len(duplicateSer>0):
    print("duplicates found!")
    print(duplicateSer)
else: 
    print("no duplicates found")

no duplicates found
